In [1]:
import torch
from supervoice_valle import SupervoceNARModel, Tokenizer
from IPython.display import Audio, display
import matplotlib.pyplot as plt
from vocos import Vocos

In [2]:
vocos = Vocos.from_pretrained("charactr/vocos-encodec-24khz")
tokenizer = Tokenizer("./tokenizer_text.model")
model = SupervoceNARModel()
checkpoint = torch.load("./output/valle-10.1000.pt", map_location = "cpu")
model.load_state_dict(checkpoint['model'])
# model = model.to(torch.float16)
# model = model.to("cuda")
model.eval()
step = checkpoint['step']
print(checkpoint['step'])

/home/steve/miniconda3/envs/gym/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


1000


In [3]:
id = "./external_datasets/libriheavy-encodec/9774/extermination_american_bison_1503_librivox_64kb_mp3/exterminationamericanbison_12_hornaday_64kb_99"
with open(id + ".txt", 'r') as file:
    text = file.read()
    text = tokenizer.encode(text)
encoded = torch.load(id + ".pt")

In [4]:
def decode(tokens):
    features = vocos.codes_to_features(tokens)
    bandwidth_id = torch.tensor([2])  # 6 kbps
    return vocos.decode(features, bandwidth_id=bandwidth_id)
display(Audio(data=decode(encoded).cpu(), rate=24000))
print(encoded.shape)

torch.Size([8, 936])


In [5]:
condition_text = text
condition_audio = encoded[:,0:75*3]
audio = encoded[:,75*3:]
predicted = [audio[0]]
for i in range(1, 8):
    p = model(
        condition_text = [condition_text], 
        condition_audio = [condition_audio],
        audio = [audio],
        codec = [i]
    )
    print(p)
    p = torch.nn.functional.softmax(p[0], dim=-1)
    print(p)
    p = torch.argmax(p, dim=-1)
    print(p)
    print(audio[i])
    predicted.append(p.cpu())
predicted = torch.stack(predicted)
predicted = torch.cat([condition_audio, predicted], dim = 1)
display(Audio(data=decode(predicted).cpu(), rate=24000))

[tensor([[4.7806, 3.1276, 3.8079,  ..., 4.3099, 4.7825, 3.6411],
        [4.8209, 3.2050, 3.8657,  ..., 4.3804, 4.8837, 3.3818],
        [4.8680, 3.3596, 3.8978,  ..., 4.3586, 4.9465, 3.3078],
        ...,
        [5.0832, 3.2837, 3.9989,  ..., 4.2656, 4.6392, 4.3881],
        [4.9680, 3.2101, 4.0441,  ..., 4.2665, 4.7470, 4.3560],
        [5.0083, 3.0129, 4.0875,  ..., 4.3014, 4.8431, 4.3358]],
       grad_fn=<SliceBackward0>)]
tensor([[0.0010, 0.0002, 0.0004,  ..., 0.0006, 0.0010, 0.0003],
        [0.0010, 0.0002, 0.0004,  ..., 0.0007, 0.0011, 0.0002],
        [0.0011, 0.0002, 0.0004,  ..., 0.0006, 0.0012, 0.0002],
        ...,
        [0.0012, 0.0002, 0.0004,  ..., 0.0005, 0.0007, 0.0006],
        [0.0010, 0.0002, 0.0004,  ..., 0.0005, 0.0008, 0.0006],
        [0.0011, 0.0001, 0.0004,  ..., 0.0005, 0.0009, 0.0005]],
       grad_fn=<SoftmaxBackward0>)
tensor([518, 518, 518, 518, 518, 424, 424, 424, 424, 424, 424, 424, 424, 518,
        518, 518, 518, 518, 518, 518, 518, 518, 518, 518

In [6]:
plt.imshow(model.positional_embedding.weight.cpu().detach().numpy(), cmap='hot', interpolation='nearest')
plt.show()

AttributeError: 'Tensor' object has no attribute 'weight'

In [ ]:
# model_t = SupervoceNARModel()
# for i in range(1, 10):
#     checkpoint = torch.load("./output/valle-05." + str(step - i * 1000) + ".pt")
#     model_t.load_state_dict(checkpoint['model'])
#     model_t = model_t.to(torch.float16)
#     model_t = model_t.to("cuda")
#     predicted = [audio[0]]
#     for i in range(1, 8):
#         p = model_t(
#             condition_text = condition_text.to("cuda"), 
#             condition_audio = condition_audio.to("cuda"),
#             audio = audio.to("cuda"),
#             codec = i
#         )
#         p = torch.nn.functional.softmax(p, dim=-1)
#         p = torch.argmax(p, dim=-1)
#         predicted.append(p.cpu())
#     predicted = torch.stack(predicted)
#     predicted = torch.cat([condition_audio, predicted], dim = 1)
#     display(Audio(data=decode(predicted).cpu(), rate=24000))